In [1]:
library(purrr)
library(magrittr)
library(tidyr)
library(tidytext)
library(dplyr)
library(stringr)
library(ggplot2)
library(tm)
library(topicmodels)
library(broom)

library(e1071)
library(caret)
library(randomForest)

Warning message:
"package 'purrr' was built under R version 3.5.2"Warning message:
"package 'magrittr' was built under R version 3.5.2"
Attaching package: 'magrittr'

The following object is masked from 'package:purrr':

    set_names

Warning message:
"package 'tidyr' was built under R version 3.5.2"
Attaching package: 'tidyr'

The following object is masked from 'package:magrittr':

    extract

Warning message:
"package 'tidytext' was built under R version 3.5.2"Warning message:
"package 'dplyr' was built under R version 3.5.2"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'stringr' was built under R version 3.5.2"Warning message:
"package 'ggplot2' was built under R version 3.5.2"Warning message:
"package 'tm' was built under R version 3.5.2"Loading required package: NLP
Warning message:
"package 'NLP' was b

In [2]:
reg <- "([^A-Za-z\\d#@']+|'[^A-Za-z\\d#@]+)"

df <- read.csv(
  "..\\..\\data\\Weekly_data_top_week.csv",
  encoding = "UTF-8",
  stringsAsFactors = FALSE)

df_words <- read.csv(
  "..\\..\\data\\Weekly_data_tokenized.csv", 
  encoding = "UTF-8",
  stringsAsFactors = FALSE)

unique_df_words <- read.csv(
  "..\\..\\data\\Weekly_data_tokenized_unique.csv", 
  encoding = "UTF-8",
  stringsAsFactors = FALSE)

metrics <- read.csv(
  "..\\..\\data\\Song_Complexity.csv",
  encoding = "UTF-8",
  stringsAsFactors = FALSE
)

df <- merge(x = df, y = metrics, by = c("X"), all.x = TRUE)
colnames(df)[colnames(df) == "Songwriter.x"] <- "Songwriter"

#final_df <- merge(x = max_week, y = df, by = c("Artists", "Name", "Weeks.on.chart"), all.x = TRUE)

unique_df_words <- unique_df_words %>%
    mutate(Pop = ifelse(str_detect(Genre, regex("pop", ignore_case = TRUE)), TRUE, FALSE)) %>%
    filter(str_detect(Genre, regex("rap|pop", ignore_case = TRUE)))

df_words <- df_words %>%
    mutate(Pop = ifelse(str_detect(Genre, regex("pop", ignore_case = TRUE)), TRUE, FALSE)) %>%
    filter(str_detect(Genre, regex("rap|pop", ignore_case = TRUE)))

df <- df %>%
    mutate(Pop = ifelse(str_detect(Genre, regex("pop", ignore_case = TRUE)), TRUE, FALSE)) %>%
    filter(str_detect(Genre, regex("rap|pop", ignore_case = TRUE)))

Warning message:
"package 'bindrcpp' was built under R version 3.5.2"

In [3]:
set.seed(1)
df$Pop <- factor(df$Pop)
df$Songwriter <- factor(df$Songwriter)

indices <- sample(df$X)
train <- df[df$X %in% indices[1:(nrow(df)*0.8)],]
test <- df[df$X %in% indices[(nrow(df)*0.8+1):nrow(df)],]

train.lab = df[df$X %in% indices[1:(nrow(df)*0.8)],]$Pop
test.lab = df[df$X %in% indices[(nrow(df)*0.8+1):nrow(df)],]$Pop

## weeks.on.chart regression

In [20]:
first <- glm(Weeks.on.chart ~ 1,
             data = train)
last <- lm(Weeks.on.chart ~ Peak.position + Pop + diversity + 
                  profanity_ratio + tf_idf_complexity + Songwriter +
                  mean_word_rarity + mean_word_length + Pop * Peak.position +
                  diversity * Pop + profanity_ratio * Pop + tf_idf_complexity * Pop + 
                  Songwriter * Pop + mean_word_rarity * Pop + mean_word_length * Pop,
             data = train)
#summary(last)

In [22]:
selected <- step(last, 
                 #scope = list(lower = first, upper = last), 
                 directions = "backwards", 
                 k = log(2983))

Start:  AIC=10257.08
Weeks.on.chart ~ Peak.position + Pop + diversity + profanity_ratio + 
    tf_idf_complexity + Songwriter + mean_word_rarity + mean_word_length + 
    Pop * Peak.position + diversity * Pop + profanity_ratio * 
    Pop + tf_idf_complexity * Pop + Songwriter * Pop + mean_word_rarity * 
    Pop + mean_word_length * Pop

                        Df Sum of Sq    RSS   AIC
- Pop:profanity_ratio    1      9.47 166482 10249
- Pop:mean_word_length   1     18.07 166490 10249
- Pop:mean_word_rarity   1     19.99 166492 10249
- Pop:tf_idf_complexity  1     32.82 166505 10250
- Pop:diversity          1    153.27 166625 10251
- Pop:Songwriter         1    332.11 166804 10254
<none>                               166472 10257
- Peak.position:Pop      1   2895.00 169367 10290

Step:  AIC=10249.22
Weeks.on.chart ~ Peak.position + Pop + diversity + profanity_ratio + 
    tf_idf_complexity + Songwriter + mean_word_rarity + mean_word_length + 
    Peak.position:Pop + Pop:diversity + Pop:

In [23]:
summary(selected)


Call:
lm(formula = Weeks.on.chart ~ Peak.position + Pop + Peak.position:Pop, 
    data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-24.816  -4.548   0.278   4.314  60.564 

Coefficients:
                       Estimate Std. Error t value Pr(>|t|)    
(Intercept)           23.149862   0.589586  39.265  < 2e-16 ***
Peak.position         -0.236878   0.009514 -24.898  < 2e-16 ***
PopTRUE                4.215696   0.697073   6.048 1.70e-09 ***
Peak.position:PopTRUE -0.073016   0.011624  -6.281 3.97e-10 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 8.394 on 2382 degrees of freedom
Multiple R-squared:  0.5448,	Adjusted R-squared:  0.5442 
F-statistic: 950.2 on 3 and 2382 DF,  p-value: < 2.2e-16


- Pop songs stay longer on the charts compared to rap.
- As the peak position increases, songs stay on the chart less.
- For pop songs with higher peak positions, they stay on the chart than rap songs.

### Peak.position regression

In [27]:
first <- glm(Peak.position ~ 1,
             data = train)
last <- lm(Peak.position ~ Weeks.on.chart + Pop + diversity + 
                  profanity_ratio + tf_idf_complexity + Songwriter +
                  mean_word_rarity + mean_word_length + Pop * Weeks.on.chart +
                  diversity * Pop + profanity_ratio * Pop + tf_idf_complexity * Pop + 
                  Songwriter * Pop + mean_word_rarity * Pop + mean_word_length * Pop,
             data = train)
#summary(last)

In [28]:
selected <- step(last, 
                 #scope = list(lower = first, upper = last), 
                 directions = "backwards", 
                 k = log(2983))

Start:  AIC=14719.7
Peak.position ~ Weeks.on.chart + Pop + diversity + profanity_ratio + 
    tf_idf_complexity + Songwriter + mean_word_rarity + mean_word_length + 
    Pop * Weeks.on.chart + diversity * Pop + profanity_ratio * 
    Pop + tf_idf_complexity * Pop + Songwriter * Pop + mean_word_rarity * 
    Pop + mean_word_length * Pop

                        Df Sum of Sq     RSS   AIC
- Pop:diversity          1       1.0 1080485 14712
- Pop:profanity_ratio    1     108.4 1080593 14712
- Pop:mean_word_rarity   1     135.9 1080620 14712
- Pop:mean_word_length   1     310.0 1080794 14712
- Pop:tf_idf_complexity  1     403.4 1080888 14713
- Pop:Songwriter         1    3127.5 1083612 14719
<none>                               1080484 14720
- Weeks.on.chart:Pop     1   13489.0 1093973 14741

Step:  AIC=14711.71
Peak.position ~ Weeks.on.chart + Pop + diversity + profanity_ratio + 
    tf_idf_complexity + Songwriter + mean_word_rarity + mean_word_length + 
    Weeks.on.chart:Pop + Pop:profan

In [29]:
summary(selected)


Call:
lm(formula = Peak.position ~ Weeks.on.chart + Pop + Songwriter + 
    Weeks.on.chart:Pop, data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-64.231 -14.922   1.385  17.105  84.521 

Coefficients:
                       Estimate Std. Error t value Pr(>|t|)    
(Intercept)            77.06064    1.03875  74.186  < 2e-16 ***
Weeks.on.chart         -2.16594    0.07375 -29.367  < 2e-16 ***
PopTRUE                -7.76107    1.29983  -5.971 2.71e-09 ***
SongwriterTrue         -5.85510    1.33664  -4.380 1.24e-05 ***
Weeks.on.chart:PopTRUE  0.43404    0.08398   5.169 2.55e-07 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 21.36 on 2381 degrees of freedom
Multiple R-squared:  0.5484,	Adjusted R-squared:  0.5477 
F-statistic: 722.9 on 4 and 2381 DF,  p-value: < 2.2e-16


- As the weeks on chart increase, the peak position of songs decrease (it means songs do better because their peak position is lower)
- Pop songs have higher peak positions than rap songs.
- Songs that contain songwriters tend to have lower peak positions
- For pop songs, as the number of weeks on the chart increase, they have lower peak positions than rap songs.